##### Deep Learing ###

1) Data Cleaning

In [23]:
from tensorflow.keras import backend as K
K.clear_session()

In [27]:
from logic.processing import load_data, preproc
import os
import string
import nltk


import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Flotchi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/Flotchi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [4]:
path = os.getcwd()

In [5]:
rpath_train = os.path.join(os.path.dirname(path),'raw_data', 'drugsComTrain_raw.csv') 
df_train = pd.read_csv(rpath_train)

In [6]:
rpath_test = os.path.join(os.path.dirname(path),'raw_data', 'drugsComTest_raw.csv') 
df_test = pd.read_csv(rpath_test)

In [7]:
def data_filter(df,top_conditions=10,uselfCount=0,min_length=30):
    
    df = df.dropna()
    condition_distribution = df['condition'].value_counts()
    top_conditions = condition_distribution.head(top_conditions)
    top_conditions_list = top_conditions.index.tolist()
    df = df[df['condition'].isin(top_conditions_list)]
    df = df[df['usefulCount']>uselfCount]
    df['review_length'] = df['review'].apply(lambda x: len(str(x).split()))
    df = df[df.review_length>=min_length]
    
    return df

In [8]:
def process(df):
    def clean(st):
        for punc in string.punctuation:
            st = st.replace(punc, '')
        ans = st.casefold().replace('\n', ' ')
        ansd = ''.join(x for x in ans if not x.isdigit())
        stop = set(stopwords.words('english'))
        tokens = word_tokenize(ansd)
        ansdd = [y for y in tokens if y not in stop]
        lemmaverb = [WordNetLemmatizer().lemmatize(word, pos='v') for word in ansdd]
        lemmanouns = [WordNetLemmatizer().lemmatize(word, pos='n') for word in lemmaverb]
        nans = ' '.join(lemmanouns)
        return nans
    df['review clean'] = df.review.apply(clean)
    return df

In [9]:
df_filter_train = data_filter(df_train)
df_filter_test = data_filter(df_test)

In [10]:
df_proc_train = process(df_filter_train)
df_proc_test = process(df_filter_test)

In [11]:
X_train = df_proc_train['review clean']
X_test = df_proc_test['review clean']
y_train = df_proc_train['condition']
y_test = df_proc_test['condition']

3) Tokenizer

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

5) Label encoding

In [13]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

5) Padding

In [14]:
max_len = 200

In [15]:
X_train_pad = pad_sequences(X_train_token, padding='post',maxlen=max_len)
X_test_pad = pad_sequences(X_test_token, padding='post',maxlen=max_len)

6.1) Model A

In [28]:
m1 = Sequential()
m1.add(Embedding(input_dim=10000, output_dim=128, input_length=200)) 
m1.add(Flatten())
m1.add(Dense(32, activation = 'relu'))
m1.add(Dense(len(y_train_encoded), activation = 'softmax'))

m1.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

In [29]:
m1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
history = m1.fit(X_train_pad, y_train_encoded, validation_split=0.2, epochs=5, batch_size=64)

Epoch 1/5


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 63888)

In [ ]:
predictions = model.predict(X_test_pad)

In [ ]:
cm = confusion_matrix(y_test_encoded, predictions)

In [ ]:
plt.figure(figsize=(10, 8))
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_train))
cm_display.plot(cmap='Blues', xticks_rotation=45)
plt.title('Confusion Matrix')
plt.show()